# **Botnet Detection in IOT devices using an Autoencoder**

## Loading Required Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


##Autoencoder

In [ ]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            layers.Dense(297, activation="relu"),
            layers.Dense(201, activation="relu"),
            layers.Dense(103, activation="relu"),
            layers.Dense(56, activation="relu"),
            layers.Dense(28, activation="relu")
        ])
        self.decoder = Sequential([
            layers.Dense(56, activation="relu"),
            layers.Dense(103, activation="relu"),
            layers.Dense(201, activation="relu"),
            layers.Dense(297, activation="sigmoid")
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

##Function to Print Stats

In [ ]:
def print_stats(data, outcome):
    print(f"Shape of data: {data.shape}")
    print(f"Detected anomalies: {np.mean(outcome)*100}%")
    print()

def load_file(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/Web_traffic_dataset", filename),
        delimiter=",",
        skip_header=1
    )

def sci_to_num(arr):
  for i in arr:
    for j in i:
      j = "{:.8f}". format(j)
  return arr

## Device 1: Baby Monitor


###Loading the Dataset

In [ ]:
data = load_file("baby_monitor.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 62ms/step - loss: 0.2181 - val_loss: 0.2163
Epoch 2/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2139 - val_loss: 0.2103
Epoch 3/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2008 - val_loss: 0.1973
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1707 - val_loss: 0.1841
Epoch 5/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1187 - val_loss: 0.1868
Epoch 6/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0692 - val_loss: 0.1930
Epoch 7/800
4/4 [==============================] - 0s 22ms/step - loss: 0.0437 - val_loss: 0.1893
Epoch 8/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0329 - val_loss: 0.1838
Epoch 9/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0300 - val_loss: 0.1824
Epoch 10/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0291 - val_loss: 0.1806
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 2: Lights

###Loading the Dataset

In [ ]:
data = load_file("lights.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 66ms/step - loss: 0.2451 - val_loss: 0.2425
Epoch 2/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2424 - val_loss: 0.2372
Epoch 3/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2355 - val_loss: 0.2242
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2192 - val_loss: 0.1958
Epoch 5/800
4/4 [==============================] - 0s 14ms/step - loss: 0.1850 - val_loss: 0.1438
Epoch 6/800
4/4 [==============================] - 0s 19ms/step - loss: 0.1260 - val_loss: 0.0774
Epoch 7/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0598 - val_loss: 0.0342
Epoch 8/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0210 - val_loss: 0.0169
Epoch 9/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0059 - val_loss: 0.0120
Epoch 10/800
4/4 [==============================] - 0s 16ms/step - loss: 0.0020 - val_loss: 0.0114
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 3: Monitor Sensor

###Loading the Dataset

In [ ]:
data = load_file("monitor_sensor.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 65ms/step - loss: 0.2169 - val_loss: 0.2131
Epoch 2/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2139 - val_loss: 0.2077
Epoch 3/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2052 - val_loss: 0.1940
Epoch 4/800
4/4 [==============================] - 0s 14ms/step - loss: 0.1836 - val_loss: 0.1657
Epoch 5/800
4/4 [==============================] - 0s 17ms/step - loss: 0.1400 - val_loss: 0.1286
Epoch 6/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0813 - val_loss: 0.1110
Epoch 7/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0420 - val_loss: 0.1107
Epoch 8/800
4/4 [==============================] - 0s 16ms/step - loss: 0.0277 - val_loss: 0.1126
Epoch 9/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0235 - val_loss: 0.1135
Epoch 10/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0230 - val_loss: 0.1126
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 4: Security Camera

###Loading the Dataset

In [ ]:
data = load_file("security_camera.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 61ms/step - loss: 0.2217 - val_loss: 0.2243
Epoch 2/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2159 - val_loss: 0.2170
Epoch 3/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1992 - val_loss: 0.1997
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1617 - val_loss: 0.1745
Epoch 5/800
4/4 [==============================] - 0s 14ms/step - loss: 0.1042 - val_loss: 0.1570
Epoch 6/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0564 - val_loss: 0.1486
Epoch 7/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0331 - val_loss: 0.1456
Epoch 8/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0252 - val_loss: 0.1434
Epoch 9/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0227 - val_loss: 0.1421
Epoch 10/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0221 - val_loss: 0.1416
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 5: Smoke Detector

###Loading the Dataset

In [ ]:
data = load_file("smoke_detector.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 76ms/step - loss: 0.2207 - val_loss: 0.2168
Epoch 2/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2115 - val_loss: 0.1996
Epoch 3/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1874 - val_loss: 0.1636
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1411 - val_loss: 0.1178
Epoch 5/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0850 - val_loss: 0.0872
Epoch 6/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0481 - val_loss: 0.0679
Epoch 7/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0267 - val_loss: 0.0549
Epoch 8/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0155 - val_loss: 0.0500
Epoch 9/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0134 - val_loss: 0.0492
Epoch 10/800
4/4 [==============================] - 0s 16ms/step - loss: 0.0131 - val_loss: 0.0502
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 6: Socket

###Loading the Dataset

In [ ]:
data = load_file("socket.csv")
data = sci_to_num(data)
train = data[150:]
test = data[:150]

print(train.shape,test.shape)

(150, 297) (150, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 59ms/step - loss: 0.2432 - val_loss: 0.2440
Epoch 2/800
4/4 [==============================] - 0s 16ms/step - loss: 0.2421 - val_loss: 0.2421
Epoch 3/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2396 - val_loss: 0.2378
Epoch 4/800
4/4 [==============================] - 0s 16ms/step - loss: 0.2338 - val_loss: 0.2280
Epoch 5/800
4/4 [==============================] - 0s 17ms/step - loss: 0.2212 - val_loss: 0.2069
Epoch 6/800
4/4 [==============================] - 0s 14ms/step - loss: 0.1943 - val_loss: 0.1648
Epoch 7/800
4/4 [==============================] - 0s 15ms/step - loss: 0.1447 - val_loss: 0.0971
Epoch 8/800
4/4 [==============================] - 0s 16ms/step - loss: 0.0779 - val_loss: 0.0331
Epoch 9/800
4/4 [==============================] - 0s 16ms/step - loss: 0.0271 - val_loss: 0.0088
Epoch 10/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0103 - val_loss: 0.0045
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (150, 297)
Detected anomalies: 0.0%



##Device 7: Thermostat

###Loading the Dataset

In [ ]:
data = load_file("thermostat.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.00001), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 61ms/step - loss: 0.2274 - val_loss: 0.2069
Epoch 2/800
4/4 [==============================] - 0s 18ms/step - loss: 0.2273 - val_loss: 0.2068
Epoch 3/800
4/4 [==============================] - 0s 16ms/step - loss: 0.2273 - val_loss: 0.2067
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2272 - val_loss: 0.2066
Epoch 5/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2271 - val_loss: 0.2066
Epoch 6/800
4/4 [==============================] - 0s 16ms/step - loss: 0.2271 - val_loss: 0.2065
Epoch 7/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2270 - val_loss: 0.2064
Epoch 8/800
4/4 [==============================] - 0s 16ms/step - loss: 0.2269 - val_loss: 0.2064
Epoch 9/800
4/4 [==============================] - 0s 19ms/step - loss: 0.2269 - val_loss: 0.2063
Epoch 10/800
4/4 [==============================] - 0s 17ms/step - loss: 0.2268 - val_loss: 0.2063
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 8: TV

###Loading the Dataset

In [ ]:
data = load_file("tv.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 70ms/step - loss: 0.1585 - val_loss: 0.1290
Epoch 2/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0443 - val_loss: 0.1272
Epoch 3/800
4/4 [==============================] - 0s 16ms/step - loss: 0.0470 - val_loss: 0.1262
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0505 - val_loss: 0.1280
Epoch 5/800
4/4 [==============================] - 0s 14ms/step - loss: 0.0532 - val_loss: 0.1280
Epoch 6/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0534 - val_loss: 0.1269
Epoch 7/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0548 - val_loss: 0.1272
Epoch 8/800
4/4 [==============================] - 0s 15ms/step - loss: 0.0550 - val_loss: 0.1318
Epoch 8: early stopping


###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 9: Watch

###Loading the Dataset

In [ ]:
data = load_file("watch.csv")
data = sci_to_num(data)
train = data[:150]
test = data[100:]

print(train.shape,test.shape)

(150, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.000001), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
4/4 [==============================] - 1s 68ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 2/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 3/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 4/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 5/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 6/800
4/4 [==============================] - 0s 15ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 7/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 8/800
4/4 [==============================] - 0s 20ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 9/800
4/4 [==============================] - 0s 14ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 10/800
4/4 [==============================] - 0s 20ms/step - loss: 0.2285 - val_loss: 0.2183
Epoch 11/800
4/4 [=

###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%



##Device 10: Water Sensor

###Loading the Dataset

In [ ]:
data = load_file("water_sensor.csv")
data = sci_to_num(data)
train = data[:100]
test = data

print(train.shape,test.shape)

(100, 297) (100, 297)


###Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
3/3 [==============================] - 1s 88ms/step - loss: 0.2104 - val_loss: 0.0041
Epoch 2/800
3/3 [==============================] - 0s 17ms/step - loss: 0.0366 - val_loss: 0.0041
Epoch 3/800
3/3 [==============================] - 0s 18ms/step - loss: 0.0370 - val_loss: 0.0041
Epoch 4/800
3/3 [==============================] - 0s 17ms/step - loss: 0.0370 - val_loss: 0.0041
Epoch 5/800
3/3 [==============================] - 0s 18ms/step - loss: 0.0370 - val_loss: 0.0041
Epoch 6/800
3/3 [==============================] - 0s 21ms/step - loss: 0.0370 - val_loss: 0.0041
Epoch 6: early stopping


###Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [test]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (100, 297)
Detected anomalies: 0.0%

